In [1]:
import numpy as np
import pandas as pd

# Fixer la graine aléatoire pour la reproductibilité
np.random.seed(42)

In [2]:
nb_obs=1000

In [3]:
data={
    "ID":[j for j in range(1,nb_obs+1)],
    "TARGET":np.random.choice([0,1], size=nb_obs),
    "segment":np.random.choice(7, size=nb_obs)
}

In [4]:
df=pd.DataFrame(data)

In [5]:
df["segment"].value_counts()

segment
0    150
5    149
6    149
4    146
2    146
1    145
3    115
Name: count, dtype: int64

#### MoC C

In [6]:
# Nombre d'échantillons bootstrap par segment
num_bootstrap_samples = 1000
moc_segment={}
# Fonction pour calculer le taux de 1 dans un échantillon bootstrap
def calculate_rate_of_ones(sample):
    return np.mean(sample)

# Boucle pour chaque segment
for segment in range(7):
    # Générer des échantillons bootstrap
    sample_size=df[df["segment"]==segment].shape[0]
    list_ids=df[df["segment"]==segment]["ID"].values
    # bootstrap_samples = [np.random.choice([0, 1], size=sample_size, replace=True) for _ in range(num_bootstrap_samples)]
    bootstrap_samples = [np.array([df.loc[df["ID"].isin(list(np.random.choice(list_ids,size=1))),"TARGET"].values for j in range(sample_size)]).flatten() for _ in range(num_bootstrap_samples)]
    
    # Calculer le taux de 1 pour chaque échantillon bootstrap
    rates_of_ones = [calculate_rate_of_ones(sample) for sample in bootstrap_samples]
    
    # Calculer le 90e centile et la moyenne
    percentile_90 = np.percentile(rates_of_ones, 90)
    mean_rate = np.mean(rates_of_ones)
    moc=percentile_90 - mean_rate
    moc_segment[segment]=moc
    # Créer un DataFrame temporaire pour le segment actuel
    temp_df = pd.DataFrame({
        'Size':[sample_size],
        'Segment': [segment],
        'Percentile_90': [percentile_90],
        'Mean_Rate': [mean_rate],
        "MoC_C": [moc]
    })
    
    print(temp_df)

   Size  Segment  Percentile_90  Mean_Rate    MoC_C
0   150        0       0.546667   0.494587  0.05208
   Size  Segment  Percentile_90  Mean_Rate     MoC_C
0   145        1       0.572414   0.518372  0.054041
   Size  Segment  Percentile_90  Mean_Rate     MoC_C
0   146        2        0.59589   0.541966  0.053925
   Size  Segment  Percentile_90  Mean_Rate     MoC_C
0   115        3       0.556522   0.494974  0.061548
   Size  Segment  Percentile_90  Mean_Rate     MoC_C
0   146        4       0.534247    0.48489  0.049356
   Size  Segment  Percentile_90  Mean_Rate     MoC_C
0   149        5       0.530201   0.476396  0.053805
   Size  Segment  Percentile_90  Mean_Rate     MoC_C
0   149        6       0.610738   0.559107  0.051631


In [8]:
moc_segment

{0: 0.05208000000000007,
 1: 0.05404137931034492,
 2: 0.05392465753424669,
 3: 0.06154782608695658,
 4: 0.04935616438356166,
 5: 0.05380536912751677,
 6: 0.0516308724832214}

In [13]:
# Nombre d'échantillons bootstrap par segment
num_bootstrap_samples = 1000
moc_segment={}
# Fonction pour calculer le taux de 1 dans un échantillon bootstrap
def calculate_rate_of_ones(sample):
    return np.mean(sample)

# Boucle pour chaque segment
for segment in range(7):
    # Générer des échantillons bootstrap
    sample_size=df[df["segment"]==segment].shape[0]
    list_ids=df[df["segment"]==segment]["ID"].values
    # bootstrap_samples = [np.random.choice([0, 1], size=sample_size, replace=True) for _ in range(num_bootstrap_samples)]
    bootstrap_samples = [df[df["segment"]==segment].sample(frac=1, replace=True)["TARGET"].values for _ in range(num_bootstrap_samples)]
    
    # Calculer le taux de 1 pour chaque échantillon bootstrap
    rates_of_ones = [calculate_rate_of_ones(sample) for sample in bootstrap_samples]
    
    # Calculer le 90e centile et la moyenne
    percentile_90 = np.percentile(rates_of_ones, 90)
    mean_rate = np.mean(rates_of_ones)
    moc=percentile_90 - mean_rate
    moc_segment[segment]=moc
    # Créer un DataFrame temporaire pour le segment actuel
    temp_df = pd.DataFrame({
        'Size':[sample_size],
        'Segment': [segment],
        'Percentile_90': [percentile_90],
        'Mean_Rate': [mean_rate],
        "MoC_C": [moc]
    })
    
    print(temp_df)

   Size  Segment  Percentile_90  Mean_Rate   MoC_C
0   150        0       0.546667   0.493867  0.0528
   Size  Segment  Percentile_90  Mean_Rate     MoC_C
0   145        1       0.572414   0.519338  0.053076
   Size  Segment  Percentile_90  Mean_Rate     MoC_C
0   146        2        0.59589   0.542555  0.053336
   Size  Segment  Percentile_90  Mean_Rate     MoC_C
0   115        3       0.547826   0.495122  0.052704
   Size  Segment  Percentile_90  Mean_Rate     MoC_C
0   146        4       0.541096   0.486384  0.054712
   Size  Segment  Percentile_90  Mean_Rate     MoC_C
0   149        5       0.530201   0.476993  0.053208
   Size  Segment  Percentile_90  Mean_Rate     MoC_C
0   149        6       0.610738   0.556899  0.053839


#### MoC A

In [ ]:
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from sklearn.cluster import KMeans

class SegmentationProblem(Problem):
    def __init__(self, n_segments, data):
        super().__init__(n_var=n_segments - 1, n_obj=2, n_constr=0, xl=np.array([0]), xu=np.array([1000]))
        self.data = data
        self.n_segments = n_segments

    def _evaluate(self, x, out, *args, **kwargs):
        kmeans = KMeans(n_clusters=self.n_segments, random_state=42).fit(self.data.reshape(-1, 1))
        labels = kmeans.labels_
        centroids = kmeans.cluster_centers_

        intra_variance = 0
        for i in range(self.n_segments):
            cluster_data = self.data[labels == i]
            if len(cluster_data) > 0:
                intra_variance += np.sum((cluster_data - centroids[i])**2)

        inter_variance = np.sum((centroids - np.mean(self.data))**2)

        # Nous répétons les valeurs des objectifs pour chaque individu dans la population
        out["F"] = np.array([[-intra_variance, inter_variance]] * x.shape[0])

# Données de test (distribution de notes entières)
np.random.seed(42)
data = np.random.randint(0, 1001, size=1000)

# Nombre de segments désirés
n_segments = 7

# Définir et résoudre le problème
problem = SegmentationProblem(n_segments, data)
algorithm = NSGA2(pop_size=100)
res = minimize(problem, algorithm, ('n_gen', 100), seed=42)

# Affichage des résultats
best_solution = res.X[np.argmin(res.F[:, 0])]
print("Seuils de segmentation optimaux :", best_solution)
